In [ ]:
# Feature values need to be formated.

Features need be represented as quantitative (preferably numeric) attributes of the thing you're sampling. They can be real world values, such as the readings from a sensor, and other discernible, physical properties. Alternatively, your features can also be calculated derivatives, such as the presence of certain edges and curves in an image, or lack thereof.

If your data comes to you in a nicely formed, numeric, tabular format, then that's one less thing for you to worry about. But there is no guarantee that will be the case, and you will often encounter data in textual or other unstructured forms. Luckily, there are a few techniques that when applied, clean up these scenarios.

#### Textual Categorical-Features

If you have a categorical feature, the way to represent it in your dataset depends on if it's ordinal or nominal. For ordinal features, map the order as increasing integers in a single numeric feature. Any entries not found in your designated categories list will be mapped to -1:

In [2]:
import pandas as pd

In [3]:
ordered_satisfaction = ['Very Unhappy', 'Unhappy', 'Neutral', 'Happy', 'Very Happy']

In [4]:
df = pd.DataFrame({'satisfaction':['Mad', 'Happy', 'Unhappy', 'Neutral']})

In [5]:
df.satisfaction = df.satisfaction.astype('category', ordered=True, categories=ordered_satisfaction).cat.codes

In [6]:
df.satisfaction

0   -1
1    3
2    1
3    2
Name: satisfaction, dtype: int8

In [8]:
df = pd.DataFrame({'vertebrates':[
            'Bird',
            'Bird',
            'Mammal',
            'Fish',
            'Amphibilian',
            'Reptil',
            'Mammal'
        ]})

In [10]:
# Method 1
df['vertebrates'] = df.vertebrates.astype('category').cat.codes
df

,vertebrates
0,1
1,1
2,3
3,2
4,0
5,4
6,3


Notice how this time, `ordered=True` was not passed in, nor was a specific ordering listed. Because of this, **Pandas encodes your nominal entries in alphabetical order**. This approach is fine for getting your feet wet, but the issue it has is that **it still introduces an ordering to a categorical list of items that inherently has none**. This may or may not cause problems for you in the future. If you aren't getting the results you hoped for, or even if you are getting the results you desired but would like to further increase the result accuracy, then a more precise encoding approach would be to **separate the distinct values out into individual boolean features**:

In [12]:
# Method 2
df = pd.get_dummies(df, columns=['vertebrates'])
df

,vertebrates_0,vertebrates_1,vertebrates_2,vertebrates_3,vertebrates_4
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,1.0,0.0


These newly created features are called boolean features because the only values they can contain are either 0 for non-inclusion, or 1 for inclusion. Pandas [.get_dummies()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) method allows you to completely replace a single, nominal feature with multiple boolean indicator features. This method is quite powerful and has many configurable options, including the ability to return a SparseDataFrame, and other prefixing options. It's benefit is that no erroneous ordering is introduced into your dataset.

#### Pure Textual Features
If you are trying to "featurize" a body of text such as a webpage, a tweet, a passage from a newspaper, an entire book, or a PDF document, creating a corpus of words and counting their frequency is an extremely powerful encoding tool. This is also known as the *Bag of Words* model, implemented with the `CountVectorizer()` method in SciKit-Learn. Even though the grammar of your sentences and their word-order are complete discarded, this model has accomplished some pretty amazing things, such as correctly identifying J.K. Rowling's writing from a blind line up of authors.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
corpus = [
    "Authman ran faster than Harry because he is an athlete.",
    "Authman and Harry ran faster and faster.",
    ]

In [16]:
cv = CountVectorizer()

In [19]:
X = cv.fit_transform(corpus)

In [18]:
cv.get_feature_names()

['an',
 'and',
 'athlete',
 'authman',
 'because',
 'faster',
 'harry',
 'he',
 'is',
 'ran',
 'than']

In [20]:
X.toarray()

array([[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 2, 0, 1, 0, 2, 1, 0, 0, 1, 0]], dtype=int64)

Some new points of interest. X is not the regular `[n_samples, n_features]` dataframe you are familiar with. Rather, it is a SciPy compressed, sparse, row matrix. SciPy is a library of mathematical algorithms and convenience functions that further extend NumPy. The reason X is now a sparse matrix instead of a classical dataframe is because even with this small example of two sentences, 11 features got created. The average English speaker knows around 8000 unique words. If each sentence were an 8000-sized vector sample in your dataframe, consisting mostly of 0's, it would be a poor use of memory.

To avoid this, SciPy implements their sparse matrices like Python implements its dictionaries: only the keys that have a value get stored, and everything else is assumed to be empty. You can always convert it to a regular Python list by using the `.toarray()` method, but this converts it to a dense array, which might not be desirable due to memory reasons. To use your compressed, spare, row matrix in Pandas, you're going to want to convert it to a Pandas SparseDataFrame. More notes on that in the Dive Deeper section.

The bag of words model has other configurable parameters, such as considering the order of words. In such implementations, pairs or tuples of successive words are used to build the corpus instead of individual words:

In [ ]:
cv.get_feature_names()

'''
expected output: 
['authman ran', 'ran faster', 'faster than', 
'than harry', 'harry because', 'because he', 
'he is', 'is an', 'an athlete', 'authman and', 
'and harry', 'harry ran', 'faster and', 'and faster']
'''

Another parameter is to have it use frequencies ant not counts. This is useful when you have documents of different lengths, so to allow direct comparisons even though the raw counts for the longer document would of course be higher. Dive deeper into the feature extraction section of SciKit-Learn's documentation to learn more about how you can best represent your textual features!

#### Refers to:
+ [Working with text](https://de.dariah.eu/tatom/working_with_text.html)

#### Graphical and Audio Features

##### Graphical Features
In addition to text and natural language processing, bag of words has successfully been applied to images by categorizing a collection of regions and describing only their appearance, ignoring any spatial structure. However this is not the typical approach used to represent images as features, and requires you come up with methods of categorizing image regions. More often used methods include:

1. Split the image into a grid of smaller areas, and attempt feature extraction at each locality. Return a combined array of all discovered. features
2. Use variable-length gradients and other transformations as the features, such as regions of high / low luminosity, histogram counts for horizontal and vertical black pixels, stroke and edge detection, etc.
3. Resize the picture to a fixed size, convert it to grayscale, then encode every pixel as an element in a uni-dimensional feature array.

Let's explore how you might go about using the third method with code:

In [ ]:
# Uses the Image module (PIL)
from scipy import misc

# Load the image up
img = misc.imread('Datasets/image.png')

# Is the image too big? Resample it down by an order of magnitude
img = img[::2, ::2]

# Scale colors from (0-255) to (0-1), then reshape to 1D array per pixel, e.g. grayscale
# If you had color images and wanted to preserve all color channels, use .reshape(-1,3)
X = (img / 255.0).reshape(-1)

# To-Do: Machine Learning with X!
#

If you're wondering what the :: is doing, that is called [extended slicing](https://docs.python.org/release/2.3.5/whatsnew/section-slices.html). Notice the `.reshape(-1)` line. This tells Pandas to take your 2D image and flatten it into a 1D array. This is an all purpose method you can use to change the shape of your dataframes, so long as you maintain the number of elements. For example reshaping a [6, 6] to [36, 1] or [3, 12], etc. Another method called `.ravel()` will do the same thing as `.reshape(-1)`, that is unravel a multi-dimensional NDArray into a one dimensional one. The reason why its important to reshape your 2D array images into one dimensional ones is because each image will represent a single sample, and SKLearn expects your dataframe to be shapes `[num_samples, num_features]`.

##### Audio Features

Audio can be encoded with similar methods as graphical features, with the caveat that your 'audio-image' is already a one-dimensional waveform data type instead of a two-dimensional array of pixels. Rather than looking for graphical gradients, you would look for auditory ones, such as the length of sounds, power and noise ratios, and histogram counts after applying filters.

In [ ]:
import scipy.io.wavfile as wavfile

sample_rate, audio_data = wavfile.read('sound.wav')
print audio_data
# To-Do: Machine Learning with audio_data!
#